In [1]:
import pandas as pd
import math

In [5]:
#df = pd.read_csv('IRIS.csv')
#df.head()


In [2]:
from collections import Counter
class DicisionTree:
  X = []  
  y = []
  Tree = []
  def __init__(self, X, y):
    self.X = X
    self.y = y
    self.Tree = []
    self.Tree = DicisionTree.Fit(self, X, y)
    print(self.Tree)
    

  def Fit(self, X, y):
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    TreeNode = []
    #----------------handling leaf nodes------------------
    if (len(X.columns) == 1):
      
      for xx in X:
        xLabel = list(set(X[xx]))
      #print('Node','<',xx,'>') 
      TreeNode.append(xx)
      for xlabel in xLabel:
        temTreeNode = []

        #print('Edge',xlabel)
        temTreeNode.append(xlabel)
        yLabel = list(set(y))
        yLabelCount = []
        for ylabel in yLabel:
          yLabelCount.append(0) 
        
        for i in range(len(X)):
          if (X[xx][i] == xlabel):
            for j in range(len(yLabel)):
              if (yLabel[j] == y[i]):
                yLabelCount[j] += 1
        
        max = 0
        loc = 0
        for i in range(len(yLabelCount)):
          if (max < yLabelCount[i]):
            max = yLabelCount[i]
            loc = i
        #print('leaf',yLabel[loc])
        temTreeNode.append(yLabel[loc])
        TreeNode.append(temTreeNode)
      return TreeNode
      #---------------------------------------------




    Gain = []
    Attribute = []
    for x in X:
      Gain.append(DicisionTree.AttributeGain(self, X[x]))
      Attribute.append(x)
    
    max = 0
    selectedAttribute = ''
    for i in range(len(Gain)):
      if (max < Gain[i]):
        max = Gain[i]
        selectedAttribute = Attribute[i]
    print()
    #print('Node','<',selectedAttribute,'>')
    TreeNode.append(selectedAttribute)

    xLabel = list(set(X[selectedAttribute]))
    
    for xlabel in xLabel:
      temTreeNode = []
      #print('Edge',xlabel)
      temTreeNode.append(xlabel)
      temDf = pd.concat([X,y],axis = 1)
      temDf = temDf[temDf[selectedAttribute] == xlabel]
      if (len(list(set(temDf[y.name]))) == 1):
        #print('leaf',list(set(temDf[y.name])))
        temTreeNode.append(list(set(temDf[y.name])))
      else:
        tem2Df = pd.concat([X,y],axis = 1)
        tem2Df = tem2Df[tem2Df[selectedAttribute] != xlabel]
        yy = tem2Df[y.name]
        XX = tem2Df.drop([y.name, selectedAttribute], axis = 1)
        temTreeNode.append(DicisionTree.Fit(self, XX, yy))
      TreeNode.append(temTreeNode)
        
    return TreeNode
      




  def AttributeGain(self, attribute):
    
    distinctValues = list(set(attribute))
    distinctValuesCount = []
    Entropy  = []
    for dv in distinctValues:
        Entropy.append(0)
        distinctValuesCount.append(0)

    for k in range(len(distinctValues)):
      distinctLable = list(set(self.y))
      distinctLableCount = []
      for dl in distinctLable:
        distinctLableCount.append(0)

      for i in range(len(attribute)):
        if (attribute[i] == distinctValues[k]):
          distinctValuesCount[k] += 1
          for j in range(len(distinctLable)):
            if (distinctLable[j] == self.y[i]):
              distinctLableCount[j] += 1
    
      for i in range(len(distinctLableCount)):
        if ((distinctLableCount[i])/sum(distinctLableCount) !=  0):
          Entropy[k] -= ((distinctLableCount[i])/sum(distinctLableCount)  * math.log2 ((distinctLableCount[i])/sum(distinctLableCount)))
      
    gain = 0
    for i in range(len(distinctValues)):
      gain -= ((distinctValuesCount[i] / sum(distinctValuesCount))  *  Entropy[i])
    
    gain += DicisionTree.CalculateEntropy(self)
    return gain




  def CalculateEntropy(self):

    #-------unique lablel------------
    distinctLable = list(set(self.y))
    #-------------------------------

    #--------calculating number of differnt unique labels----------
    distinctLableCount = []
    for d in distinctLable:
      distinctLableCount.append(0)


    for y in self.y:     
      for i in range(len(distinctLable)):
        if (y == distinctLable[i]):
          distinctLableCount[i] += 1
    #------------------------------------------------------------

    #-------------------------------------------calculating entoropy----------------------
    entropy = 0

    for i in range(len(distinctLableCount)):
      entropy -= ((distinctLableCount[i])/sum(distinctLableCount)  * math.log2 ((distinctLableCount[i])/sum(distinctLableCount)))

    #print(entropy)
    return entropy
    #-------------------------------------------------------------------------------------------

  def Search(self, Tree , SarchValue):

    for sv in SarchValue:
      if (Tree[0] == sv):
        i = 1
        while (i < len(Tree)):
          if (Tree[i][0] == SarchValue[sv][0]):
            print(Tree[i][1])
            return True
          i +=1
        i = 1
        while (i < len(Tree)):
          for svv in SarchValue:
            if (Tree[i][1] == svv ):
              if (DicisionTree.Search(self, Tree[i], SarchValue) == True):
                return True
          i += 1
        



    


In [3]:
df = pd.DataFrame({'Gender': ['male', 'male', 'female', 'female','male','male','female','female','male','female'], 
                   'Ownership Car': ['0','1','1','0','1','0','1','1','2','2'],
                   'Travel Cost': ['cheap','cheap','cheap','cheap','cheap','standerd','standerd','expensive','expensive','expensive'],
                   'Income level': ['low','medium','medium','low','medium','medium','medium','high','medium','high'],
                   'Transport': ['bus', 'bus','train','bus','bus','train','train','car','car','car']})
y = df['Transport']
X = df.drop(['Transport'], axis = 1)
print(df)

SearchValue = pd.DataFrame({'Gender': ['male'], 
                           'Ownership Car': ['0'],
                           'Travel Cost': ['expensive'],
                           'Income level': ['medium']})

DT = DicisionTree(X, y)

DicisionTree.Search(DT,DT.Tree, SearchValue)
#for d in df:
  #print(df[d])
#DicisionTree.AttributeGain(DT, df['Travel Cost'])
#DicisionTree.Fit(DT)



   Gender Ownership Car Travel Cost Income level Transport
0    male             0       cheap          low       bus
1    male             1       cheap       medium       bus
2  female             1       cheap       medium     train
3  female             0       cheap          low       bus
4    male             1       cheap       medium       bus
5    male             0    standerd       medium     train
6  female             1    standerd       medium     train
7  female             1   expensive         high       car
8    male             2   expensive       medium       car
9  female             2   expensive         high       car



['Travel Cost', ['standerd', ['train']], ['cheap', ['Ownership Car', ['1', ['Gender', ['male', ['Income level', ['high', 'car']]], ['female', ['car']]]], ['0', ['train']], ['2', ['car']]]], ['expensive', ['car']]]
['car']


True